In [118]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
import numpy as np
import soundfile as sf
from pathlib import Path
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel
import IPython.display as ipd

[NeMo I 2022-07-12 10:35:58 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-12 10:35:58 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-12 10:35:58 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 10:36:02 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-12 10:36:02 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-12 10:36:02 features:200] PADDING: 0


[NeMo W 2022-07-12 10:36:02 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-12 10:36:02 features:200] PADDING: 0
[NeMo I 2022-07-12 10:36:04 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [174]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

In [175]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-07-12 15:31:37 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-07-12 15:31:37 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-07-12 15:31:37 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 15:31:38 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-07-12 15:31:38 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-07-12 15:31:38 features:200] PADDING: 16
[NeMo I 2022-07-12 15:31:39 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [205]:
# For original audio training
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-12 17:20:10 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-12 17:20:11 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-12 17:20:12 exp_manager:287] Experiments will be logged at


Epoch 24: 100%|██████████| 26/26 [00:21<00:00,  1.19it/s, loss=4.18, v_num=0-12]Traceback (most recent call last):
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Epoch 24: 100%|██████████| 26/26 [00:22<00:00,  1.14it/s, loss=4.18, v_num=0-12]
                                                                                Epoch 24, global step 625: 'v_loss' reached 1.19441 (best 1.19441), saving model to '/home/dgxuser/Desktop/kailin/S91_no_mixing/FastPitch/2022-07-12_17-20-12/checkpoints/FastPitch--v_loss=1.1944-epoch=24.ckpt' as top 3
Epoch 39: 100%|██████████| 25/25 [00:13<00:00,  1.88it/s, loss=4.08, v_num=0-12]


In [208]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [209]:
speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

S91_no_mixing/FastPitch/2022-07-12_17-20-12/checkpoints/FastPitch--v_loss=1.1944-epoch=24-last.ckpt
[NeMo I 2022-07-12 17:32:30 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-12 17:33:09 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-12 17:33:10 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_d

[NeMo I 2022-07-12 17:33:10 features:200] PADDING: 1
[NeMo I 2022-07-12 17:33:10 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-12 17:33:10 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-12 17:33:10 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 17:33:13 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-12 17:33:13 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-12 17:33:13 features:200] PADDING: 0


[NeMo W 2022-07-12 17:33:13 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-12 17:33:13 features:200] PADDING: 0
[NeMo I 2022-07-12 17:33:15 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [ ]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_original/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_original.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [ ]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

In [ ]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

In [210]:
def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

# Get records from the training manifest
records = []
train_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json'
with open(train_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpect/S91/trainSpect')
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        #if "normalized_text" in r:
            #text = spec_model.parse(r["normalized_text"], normalize=False)
        #else:
            #text = spec_model.parse(r['text'])
        
        text = spec_model.parse(r['text'])
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

In [211]:
# Get records from the validation manifest
records = []
val_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json'
with open(val_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpect/S91/valSpect')
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        #if "normalized_text" in r:
            #text = spec_model.parse(r["normalized_text"], normalize=False)
        #else:
            #text = spec_model.parse(r['text'])
            
        text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_val.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

In [219]:
!(python hifigan_finetune.py \
  --config-name=hifigan.yaml \
  model.train_ds.dataloader_params.batch_size=32 \
  model.max_steps=1000 \
  ~model.optim.sched \
  model.optim.lr=0.00001 \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_val.json \
  exp_manager.exp_dir=S91_hifigan_ft \
  +init_from_pretrained_model=tts_hifigan \
  trainer.check_val_every_n_epoch=10 \
  model/train_ds=train_ds_finetune \
  model/validation_ds=val_ds_finetune)

[NeMo W 2022-07-12 18:01:12 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-12 18:01:13 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-07-12 18:01:13 nemo_logging:349] /home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'hifigan.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_co

[NeMo W 2022-07-12 18:01:27 nemo_logging:349] /home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (19) is smaller than the logging interval Trainer(log_every_n_steps=100). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
      rank_zero_warn(
    
Epoch 0:   0%|                                           | 0/19 [00:00<?, ?it/s][W reducer.cpp:1289] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator(

In [220]:
def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        model_name="HifiGan"
    ):    
    
    d = f"S{new_speaker_id}_hifigan_ft"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [221]:
speaker_id = 91
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id)
print(last_ckpt)
vocoder = HifiGanModel.load_from_checkpoint(last_ckpt)
vocoder = vocoder.eval().cuda()

S91_hifigan_ft/HifiGan/2022-07-12_18-01-13/checkpoints/HifiGan--val_loss=1.2017-epoch=19-last.ckpt


[NeMo W 2022-07-12 18:14:48 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json
      sample_rate: 22050
      n_segments: 8192
      max_duration: null
      min_duration: 0.75
      load_precomputed_mel: true
      hop_length: 256
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 32
      num_workers: 4
      pin_memory: true
    
[NeMo W 2022-07-12 18:14:48 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collecti

[NeMo I 2022-07-12 18:14:48 features:200] PADDING: 0
[NeMo I 2022-07-12 18:14:48 features:208] STFT using exact pad
[NeMo I 2022-07-12 18:14:48 features:200] PADDING: 0
[NeMo I 2022-07-12 18:14:48 features:208] STFT using exact pad


In [222]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_ft/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_ft.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [223]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [224]:
error = wer(transcript, predicted)
print('Word Error Rate (FineTuned): ' + str(error))

Word Error Rate (FineTuned): 0.822690099182182


In [188]:
vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

[NeMo I 2022-07-12 15:58:36 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-12 15:58:36 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-12 15:58:36 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 15:58:39 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-12 15:58:39 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-12 15:58:39 features:200] PADDING: 0


[NeMo W 2022-07-12 15:58:39 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-12 15:58:39 features:200] PADDING: 0
[NeMo I 2022-07-12 15:58:41 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [189]:
from pydub import AudioSegment
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_resampled/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_resampled.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")
    
      # resample at 16kHz
      sound = AudioSegment.from_file(file_dir, format='wav', frame_rate=22050)
      sound = sound.set_frame_rate(16000)
      file_dir = audio_dir + str(n) + '_resampled.wav'
      sound.export(file_dir, format='wav')

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [190]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [191]:
error = wer(transcript, predicted)
print('Word Error Rate (Resampled): ' + str(error))

Word Error Rate (Resampled): 0.21211066643466156
